In [ ]:
#Drive connection
!sudo echo -ne '\n' | sudo add-apt-repository ppa:alessandro-strada/ppa >/dev/null 2>&1 # note: >/dev/null 2>&1 is used to supress printing
!sudo apt update >/dev/null 2>&1
!sudo apt install google-drive-ocamlfuse >/dev/null 2>&1
!google-drive-ocamlfuse
!sudo apt-get install w3m >/dev/null 2>&1 # to act as web browser
!xdg-settings set default-web-browser w3m.desktop >/dev/null 2>&1 # to set default browser
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir "My Drive"
!google-drive-ocamlfuse "/content/gdrive/My Drive"

/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force&state=VNLsABJxqoV-I5k01brcGnb5QuKKQhD3S8w-jjzSMpc'
/bin/sh: 1: firefox: not found
/bin/sh: 1: google-chrome: not found
/bin/sh: 1: chromium-browser: not found
/bin/sh: 1: open: not found
Cannot retrieve auth tokens.
Failure("Error opening URL:https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.

In [ ]:
#install the necessary Packages
!pip install noisereduce
!pip install  pydub
!pip install wavio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import noisereduce as nr
import librosa
import librosa.display
from pydub import AudioSegment
import wavio as wv

# Define the Meta_Features of the signal audio

In [ ]:
#Meta-features directory
DIR ="/content/gdrive/My Drive/Meta_Asr/Data_Annotation/Meta_Features_Data/"

#define Meta Feature
META_FEATURES={
    0:'MFCC',
    1:'spectral_centroid',
    2:'spectral_bandwidth',
    3: 'zcr',
    4:'Length',
    5:'Duration',
    6:'Sample_Rate'}


#Extract the Meta-Features for Raw Data

In [ ]:

#configuration setup
def setup(filename):
  try:
    os.mkdir(filename)
  except OSError:
    pass


#load data
def load_data(path):
   df = pd.read_csv(path,index_col=0)
   return df





#extract the statistical features from complex features
def extract_statistical_features(audio_feature):
    max_value = np.max(audio_feature)
    min_value = np.min(audio_feature)
    mean_value = np.mean(audio_feature)
    median_value = np.median(audio_feature)
    q1_value, q3_value = np.quantile(audio_feature, [.25, .75])
    list_statistical_features = [max_value, min_value, mean_value, median_value, q1_value, q3_value]
    return list_statistical_features

def extract_simplefeatures(input_voice):
  y, sr = librosa.load(input_voice)
  length=sig_length = len(y)
  duration=round(y.shape[0] / float(sr), 2)
  return [length,duration,sr]

#signal audio Pre-Process
def audio_complex_feauters(input_voice):
    y, sr = librosa.load(input_voice)
    #extract mfcc
    mfcc = librosa.feature.mfcc(y=y,n_mfcc=13)
    spectral_centroid = librosa.feature.spectral_centroid(y=y)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y)
    zcr = librosa.feature.zero_crossing_rate(y=y)
    feature_list = [mfcc,spectral_centroid,spectral_bandwidth,zcr]
    data = [extract_statistical_features(feature) for feature in feature_list]
    return data


#extract features from raw data
def extract_MetaFeatures(df):
  list_complexFeatures = list()
  list_simpleFeatures = list()
  for i  in range(len(df['Audio'])):
    list_simpleFeatures.append(extract_simplefeatures(df['Audio'][i]))
    list_complexFeatures.append(audio_complex_feauters(df['Audio'][i]))
  return list_simpleFeatures,list_complexFeatures


#create MetaFeatures data from raw data
def create_MetaFeatures_RawData(df,list_simpleFeatures,list_complexFeatures,data_name="Raw_MetaFeatures_Data",feature_names=META_FEATURES):
  # Define the list simple features columns
  sp_col=[val for key, val in META_FEATURES.items() if int(key) >= 4 and int(key) <= 6]
  # Define the list complex features columns
  comp_col=[val for key, val in META_FEATURES.items() if int(key) >= 0 and int(key) <= 3]
  # Create a new dataframe with simple features columns
  for j, name in enumerate(sp_col):
    df[name]=[feature[j] for idx, feature in enumerate(list_simpleFeatures)]
  # Create columns for complex features
  for i,name in enumerate(comp_col):
    cols = [f'{name}_{stat}' for stat in ['max', 'min', 'mean', 'median', 'q1', 'q3']]
    for j, stat in enumerate(cols):
      df[stat]=[feature[j] for data in list_complexFeatures for idx, feature in enumerate(data) if idx == i]
  #Save the Meta_Features data in csv file
  df.to_csv(DIR+data_name+'.csv',index=0)
  return df


def createMetaFeaturesData(path):
  #load data
  df2 = load_data(path)
  #extract MetaFeatures from preprocessed audios
  list_simpleFeatures,list_complexFeatures=extract_MetaFeatures(df2)
  #Create MetaFeatures data and save it in csv format
  new_df = create_MetaFeatures_RawData(df2,list_simpleFeatures,list_complexFeatures)
  print('***MetaFeaturesRawData created with sucess***')







# Extract the Meta-Features for PreProcessed Data

In [ ]:
#load data
def load_data(path):
   df = pd.read_csv(path,index_col=0)
   return df

#preprocessing signal audio
def audio_preprocessing(input_audio,sr=16000):
  # Load the audio file
  y, sr = librosa.load(input_audio)
  # Estimate the noise profile using a portion of the audio file
  noise_profile = nr.reduce_noise(y,sr)
  # Apply spectral subtraction to obtain a clean signal
  clean_signal = y - noise_profile
  # pre-emphasis
  audio = librosa.effects.preemphasis(clean_signal)
  #Normalizing audio
  norm_audio = librosa.util.normalize(audio)
  return norm_audio


#extract simple features  from the resampling audios
def extract_simplefeatures(preprocessed_audio,sr=16000):
  list_sp_features = list()
  length=sig_length = len(preprocessed_audio)
  duration=round(preprocessed_audio.shape[0] / float(sr), 2)
  return [sr,length,duration]



def audio_complex_feauters(preprocessed_audio):
    mfcc = librosa.feature.mfcc(y=preprocessed_audio,n_mfcc=13)
    spectral_centroid = librosa.feature.spectral_centroid(y=preprocessed_audio)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=preprocessed_audio)
    zcr = librosa.feature.zero_crossing_rate(y=preprocessed_audio)
    feature_list = [mfcc, spectral_centroid , spectral_bandwidth, zcr]
    data = [extract_statistical_features(feature) for feature in feature_list]
    return data


#extract the statistical fcomplex features from Pre_processed audio
def extract_statistical_features(audio_feature):
    max_value = np.max(audio_feature)
    min_value = np.min(audio_feature)
    mean_value = np.mean(audio_feature)
    median_value = np.median(audio_feature)
    q1_value, q3_value = np.quantile(audio_feature, [.25,.75])
    list_statistical_features = [max_value, min_value, mean_value, median_value, q1_value, q3_value]
    return list_statistical_features

#extract the both simple and complex feautures from preprocessed Audios
def extract_Preprocessed_MetaFeatures(df):
  list_complexFeatures = list()
  list_simpleFeatures = list()
  rmv_index=list()
  for i  in range(len(df['Audio'])):
    try:
      audio_frames = audio_preprocessing(df['Audio'][i])
      #extract simple features from preprocessed audio data
      list_simpleFeatures.append(extract_simplefeatures(audio_frames))
      list_complexFeatures.append(audio_complex_feauters(audio_frames))
    except:
      rmv_index.append(i)
      continue
  return list_simpleFeatures,list_complexFeatures,rmv_index

#create MetaFeatures data from preprocessed data
def create_MetaFeatures_PreprocessedData(df,list_simpleFeatures,list_complexFeatures,rmv_index,data_name="PreProcessed_MetaFeatures_Data",feature_names=META_FEATURES):
  # Define the list simple features columns
  sp_col=[val for key, val in META_FEATURES.items() if int(key) >= 4 and int(key) <= 6]
  # Define the list complex features columns
  comp_col=[val for key, val in META_FEATURES.items() if int(key) >= 0 and int(key) <= 3]
  # Create a new dataframe with simple features columns
  #remove the unprocessed Audios from data
  df2 = df.drop(rmv_index)
  for j, name in enumerate(sp_col):
    df2[name]=[feature[j] for idx, feature in enumerate(list_simpleFeatures)]

  # Create satistical  columns for each complex features
  for i,name in enumerate(comp_col):
    cols = [f'{name}_{stat}' for stat in ['max', 'min', 'mean', 'median', 'q1', 'q3']]
    for j, stat in enumerate(cols):
      df2[stat]=[feature[j] for data in list_complexFeatures for idx, feature in enumerate(data) if idx == i]
  #Save the Meta_Features data in csv file
  df2.to_csv(DIR+data_name+'.csv',index=0)
  return df2

def create_PreprocessedMetaFeaturesData(path):
  #load data
  df = load_data(path)
  #extract MetaFeatures from preprocessed audios
  list_simpleFeatures,list_complexFeatures,rmv_index = extract_Preprocessed_MetaFeatures(df)
  #Create MetaFeatures data and save it in csv format
  new_df = create_MetaFeatures_PreprocessedData(df,list_simpleFeatures,list_complexFeatures,rmv_index)


In [ ]:
setup(DIR)
#path of the audi data collection
path="/content/gdrive/My Drive/Meta_Asr/Data_Annotation/Audio_Data/data_audio.csv"
#load data
df = load_data(path)
# create feature raw data
raw_df=createMetaFeaturesData(path)
# create feature Preprocesssed data
preprocessed_df= create_PreprocessedMetaFeaturesData(path)


***Start Preprocessed Meta_Features Data creation***


/usr/local/lib/python3.10/dist-packages/noisereduce/noisereduce.py:306: RuntimeWarning: invalid value encountered in true_divide
  sig_mult_above_thresh = (abs_sig_stft - sig_stft_smooth) / sig_stft_smooth
